In [1]:
# import sys
# sys.path.insert(0, '..')
from semantic_mpc_interface import (
    LoadModel,
    get_thermostat_data,
    HPFlexSurvey,
    convert_units,
    SHACLHandler,
    inline_shapes
    # add_connection
)
from buildingmotif.namespaces import BRICK, RDF, S223
from rdflib import URIRef
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library
import csv
from pyshacl.rdfutil import clone

# SELECT ONTOLOGY 
ontology = 's223'

# base path
base_path = f'{ontology}-example/bldg1'

CRITICAL:root:Install the 'bacnet-ingress' module, e.g. 'pip install buildingmotif[bacnet-ingress]'


# Testing the Model Builder

ModelBuilder has been deprecated in favor of just survey based workflow. May be readded in the future

# Testing the Survey Workflow

In [2]:
# Please disregard excessive outputs (logging and warnings)
# TODO: Figure out how to configure these in building motif 
import logging
logging.disable(logging.CRITICAL)
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Creating survey, allow overwrite if there is something already there
s = HPFlexSurvey(base_path.split('/')[0],base_path.split('/')[1],'.', overwrite=True, ontology=ontology,
                 template_dict = {'zone': 'hvac-zone',
                                  "space": "space",
                                  "hvac": "hp-rtu",
                                  "tstat": "tstat",
                                  "window": "window",
                                  "site": "site" })

# Generating a simple building structure that prefills csv files. 
s.easy_config(zone_space_window_list=[(2,2),(1,2),(1,3)])
# Now check test_site/test_build to look at survey files


removing dependency:  space
removing dependency:  window
removing dependency:  hp-rtu
removing dependency:  tstat
values: dict_values([])
values: dict_values(['area-value'])
values: dict_values(['heating_capacity-value', 'cooling_capacity-value', 'cooling_COP-value', 'heating_COP-value'])
values: dict_values(['setpoint_deadband-value', 'active-value', 'resolution-value', 'tolerance-value', 'stage_count-value'])
values: dict_values(['area-value', 'azimuth-value', 'tilt-value'])
values: dict_values(['latitude-value', 'noaastation-value', 'timezone-value', 'longitude-value'])
{'site_id': 's223-example', 'hvac_type': 'hp-rtu', 'variable_params': {'hvac-zone': {}, 'space': {'area': '<name>-area'}, 'hp-rtu': {'heating_COP': '<name>-heating_COP', 'cooling_capacity': '<name>-cooling_capacity', 'cooling_COP': '<name>-cooling_COP', 'heating_capacity': '<name>-heating_capacity'}, 'tstat': {'tolerance': '<name>-tolerance', 'stage_count': '<name>-stage_count', 'resolution': '<name>-resolution', 'se

In [4]:
# Will just fill the columns programmatically for testing, csv should be filled out otherwise
import sys
sys.path.insert(0,'../examples')
from example_prefill_usage import prefill_csv_survey

In [5]:
# prefill_csv_survey(base_path)

In [6]:
# Reading csv
s.read_csv()

Expanded CSV shape: (6, 5)
New columns added: []
Expanded CSV shape: (4, 4)
New columns added: ['area']
Expanded CSV shape: (3, 13)
New columns added: ['heating_COP', 'cooling_capacity', 'cooling_COP', 'heating_capacity']
Expanded CSV shape: (3, 14)
New columns added: ['tolerance', 'stage_count', 'resolution', 'setpoint_deadband', 'active']
Expanded CSV shape: (7, 10)
New columns added: ['azimuth', 'area', 'tilt']
Expanded CSV shape: (1, 9)
New columns added: ['noaastation', 'timezone', 'longitude', 'latitude']


# Testing SHACL Generation and Validation

In [7]:
og = clone.clone_graph(s.graph)
# Create handler
handler = SHACLHandler(ontology=ontology)

# Generate shapes
handler.generate_shapes()

# Save shapes
handler.save_shapes(f'{base_path}/shapes.ttl')
# also save more human readable shapes
inline_shapes(handler.shapes_graph).serialize(f"{base_path}/inlined_shapes.ttl")

# Run inference on model
inferred_graph = handler.infer(s.graph)

inferred_graph.serialize(f'{base_path}/reasoned.ttl', format = 'ttl')
# if not validation_result.valid:
#     print("Validation failed:")
#     print(validation_result.report_string)

<Graph identifier=52184bea-5b17-49a1-9c4d-58a2743b30dc (<class 'rdflib.graph.Graph'>)>

In [8]:
# lots of new inferred information
(inferred_graph-og).print()

@prefix ns1: <urn:hpflex/shapes#> .

<urn:hpflex/s223-example#hvac_1> a ns1:hp-rtu ;
    ns1:air-connected-to <urn:hpflex/s223-example#zone_1> ;
    ns1:has-point <urn:hpflex/s223-example#hvac_1-cooling_COP>,
        <urn:hpflex/s223-example#hvac_1-cooling_capacity>,
        <urn:hpflex/s223-example#hvac_1-heating_COP>,
        <urn:hpflex/s223-example#hvac_1-heating_capacity> .

<urn:hpflex/s223-example#hvac_2> a ns1:hp-rtu ;
    ns1:air-connected-to <urn:hpflex/s223-example#zone_2> ;
    ns1:has-point <urn:hpflex/s223-example#hvac_2-cooling_COP>,
        <urn:hpflex/s223-example#hvac_2-cooling_capacity>,
        <urn:hpflex/s223-example#hvac_2-heating_COP>,
        <urn:hpflex/s223-example#hvac_2-heating_capacity> .

<urn:hpflex/s223-example#hvac_3> a ns1:hp-rtu ;
    ns1:air-connected-to <urn:hpflex/s223-example#zone_3> ;
    ns1:has-point <urn:hpflex/s223-example#hvac_3-cooling_COP>,
        <urn:hpflex/s223-example#hvac_3-cooling_capacity>,
        <urn:hpflex/s223-example#hvac_3-

# Testing get Metadata

In [9]:
# still working on loader, will clean up class, but functionality about right
loader = LoadModel(f"{base_path}/reasoned.ttl", 
                   template_dict={'sites':'site', 'zones': 'hvac-zone'}, 
                   ontology = ontology)
site_info = loader.get_all_building_objects()

In [12]:
df = loader._get_dataframe()

In [95]:
from semantic_mpc_interface.namespaces import * 
from semantic_mpc_interface.utils import * 

main_entity_col = 'name'

# Mapping columns to templates (which are also HPFS types)
entity_types = {}
attr_types = {}

# Type of entity and types of related attributes
entity_attr_types = {}
entity_entity_types = {}

# # May delete, not sure if I need this
# entity_class_relation = {}
value_templates,entity_templates = get_template_types(ontology='s223')

# mapping entity cols to related point cols in df
entity_attr_cols = {}

# mapping entity cols to related entity cols in df
entity_entity_cols = {}
row = df.iloc[0]
# get all the entity types
for col, entity in row.items():
    for p, o in loader.g.predicate_objects(entity):
        # o is entity class
        if p == A and (loader.g.compute_qname(o)[1]) == URIRef(HPFS):
            entity_type = get_uri_name(loader.g,o)
            if entity_type in entity_templates:
                # getting entity types
                entity_types[col] = entity_type
            if entity_type in value_templates:
                # getting value types
                attr_types[col] = entity_type
            continue
        # o is an attribute of entity
        if p == HPFS['has-point']:
            # getting entity attr cols
                if col not in entity_attr_cols.keys():
                    entity_attr_cols[col] = set()
                # Should always be a single value, may have to validate this
                col_names = row.index[row == o]
                if len(col_names) != 1:
                    print(f'incorrect amount of columns returned for {o}')
                else:
                    entity_attr_cols[col].add(col_names.values[0])
        for p2, o2 in loader.g.predicate_objects(o):
            # getting entity relations if o is another defined entity
            if p2 == A and (loader.g.compute_qname(o2)[1]) == URIRef(HPFS):
                if o2 in [ HPFS[val] for val in entity_templates ]:
                    if col not in entity_entity_cols:
                        entity_entity_cols[col] = set()
                    # no self relations allowed
                    col_names = row.index[row == o]
                    if len(col_names) != 1:
                        print(f'incorrect amount of columns returned for {o}')
                    else:
                        col_value = col_names.values[0]
                        if col_value != col:
                            entity_entity_cols[col].add(col_value)

#change sets to iterables
entity_attr_cols = {entity:list(attrs) for entity, attrs in entity_attr_cols.items()}
entity_entity_cols = {entity:list(other_entities) for entity, other_entities in entity_entity_cols.items()}

# reshape direction relationships to make direction from main template of focus
reverse_related_to = []
reverse_related_to_types = []
for entity, entities in entity_entity_cols.items():
    if main_entity_col in entities:
        reverse_related_to.append(entity)
        entities.remove(main_entity_col)
        reverse_related_to_types.append(entity_templates)
entity_entity_cols[main_entity_col] += reverse_related_to

# creating type dictionaries to dynamically instantiate classes
entity_attr_types = { 
    entity_types[entity_col]: [
        attr_types[val_col] for val_col in val_cols
    ] 
    for entity_col, val_cols in entity_attr_cols.items()
}
entity_entity_types = { 
    entity_types[entity_col]: [
        entity_types[other_entity_col] for other_entity_col in other_entity_cols
    ] 
    for entity_col, other_entity_cols in entity_entity_cols.items()
}


incorrect amount of columns returned for urn:hpflex/s223-example#tstat_zone_3-tolerance
incorrect amount of columns returned for urn:hpflex/s223-example#tstat_zone_3-resolution


In [96]:
display(entity_entity_cols)
display(entity_attr_cols)
display(entity_entity_types)
display(entity_attr_types)


{'hvac': [], 'name': ['window', 'space', 'hvac', 'tstat'], 'tstat': []}

{'hvac': ['hvac_cooling_COP',
  'hvac_heating_capacity',
  'hvac_cooling_capacity',
  'hvac_heating_COP'],
 'space': ['space_area'],
 'tstat': ['tstat_stage_count', 'tstat_active', 'tstat_setpoint_deadband'],
 'window': ['window_azimuth', 'window_tilt', 'window_area']}

{'hp-rtu': [],
 'hvac-zone': ['window', 'space', 'hp-rtu', 'tstat'],
 'tstat': []}

{'hp-rtu': ['cooling-COP',
  'heating-capacity',
  'cooling-capacity',
  'heating-COP'],
 'space': ['area'],
 'tstat': ['tstat-stage_count', 'tstat-active', 'tstat-deadband'],
 'window': ['azimuth', 'tilt', 'area']}

In [ ]:
print(site_info)

{'sites': [site(name='urn:hpflex/s223-example#s223-example', timezone=Value(value=, unit='None'), noaastation=Value(value=, unit='None'), longitude=Value(value=, unit='http://qudt.org/vocab/unit/Degree'), latitude=Value(value=, unit='http://qudt.org/vocab/unit/Degree'), )], 'zones': [hvac_zone(name='urn:hpflex/s223-example#zone_3', , windows=1, spaces=1, hp_rtus=1, tstats=1), hvac_zone(name='urn:hpflex/s223-example#zone_2', , windows=1, spaces=1, hp_rtus=1, tstats=1), hvac_zone(name='urn:hpflex/s223-example#zone_1', , windows=2, spaces=2, hp_rtus=1, tstats=1)]}


In [ ]:
zone = site_info['zones'][0]
print(zone)

hvac_zone(name='urn:hpflex/s223-example#zone_3', , windows=1, spaces=1, hp_rtus=1, tstats=1)


In [ ]:
zone.spaces

[space(name='urn:hpflex/s223-example#space_3_1', area=Value(value=, unit='http://qudt.org/vocab/unit/FT2'), )]

In [ ]:
[ zone.windows for zone in site_info['zones'] ]# [0]# .area.name

[[window(name='urn:hpflex/s223-example#window_3_3', tilt=Value(value=, unit='http://qudt.org/vocab/unit/Degree'), azimuth=Value(value=, unit='http://qudt.org/vocab/unit/Degree'), area=None, )],
 [window(name='urn:hpflex/s223-example#window_2_2', tilt=Value(value=, unit='http://qudt.org/vocab/unit/Degree'), azimuth=Value(value=, unit='http://qudt.org/vocab/unit/Degree'), area=None, )],
 [window(name='urn:hpflex/s223-example#window_1_2', tilt=Value(value=, unit='http://qudt.org/vocab/unit/Degree'), azimuth=Value(value=, unit='http://qudt.org/vocab/unit/Degree'), area=None, ),
  window(name='urn:hpflex/s223-example#window_1_1', tilt=Value(value=, unit='http://qudt.org/vocab/unit/Degree'), azimuth=Value(value=, unit='http://qudt.org/vocab/unit/Degree'), area=None, )]]

In [ ]:
zone.tstats[0]

tstat(name='urn:hpflex/s223-example#tstat_zone_3', tstat_tolerance=Value(value=, unit='http://qudt.org/vocab/unit/DEG_F'), tstat_stage_count=Value(value=, unit='http://qudt.org/vocab/unit/NUM'), tstat_deadband=Value(value=, unit='http://qudt.org/vocab/unit/DEG_F'), tstat_resolution=None, tstat_active=Value(value=, unit='http://qudt.org/vocab/unit/NUM'), )

In [ ]:
from semantic_mpc_interface import query_to_df
query = 'PREFIX brick: <https://brickschema.org/schema/Brick#>\nPREFIX csvw: <http://www.w3.org/ns/csvw#>\nPREFIX dc: <http://purl.org/dc/elements/1.1/>\nPREFIX dcat: <http://www.w3.org/ns/dcat#>\nPREFIX dcmitype: <http://purl.org/dc/dcmitype/>\nPREFIX dcterms: <http://purl.org/dc/terms/>\nPREFIX dcam: <http://purl.org/dc/dcam/>\nPREFIX doap: <http://usefulinc.com/ns/doap#>\nPREFIX foaf: <http://xmlns.com/foaf/0.1/>\nPREFIX geo: <http://www.opengis.net/ont/geosparql#>\nPREFIX odrl: <http://www.w3.org/ns/odrl/2/>\nPREFIX org: <http://www.w3.org/ns/org#>\nPREFIX prof: <http://www.w3.org/ns/dx/prof/>\nPREFIX prov: <http://www.w3.org/ns/prov#>\nPREFIX qb: <http://purl.org/linked-data/cube#>\nPREFIX schema: <https://schema.org/>\nPREFIX sh: <http://www.w3.org/ns/shacl#>\nPREFIX skos: <http://www.w3.org/2004/02/skos/core#>\nPREFIX sosa: <http://www.w3.org/ns/sosa/>\nPREFIX ssn: <http://www.w3.org/ns/ssn/>\nPREFIX time: <http://www.w3.org/2006/time#>\nPREFIX vann: <http://purl.org/vocab/vann/>\nPREFIX void: <http://rdfs.org/ns/void#>\nPREFIX wgs: <https://www.w3.org/2003/01/geo/wgs84_pos#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\nPREFIX xml: <http://www.w3.org/XML/1998/namespace>\nPREFIX ns1: <urn:___param___#>\nPREFIX ns2: <http://data.ashrae.org/standard223#>\nPREFIX ns3: <http://qudt.org/vocab/quantitykind/>\nPREFIX ns4: <http://qudt.org/schema/qudt/>\nPREFIX ns5: <http://qudt.org/vocab/unit/>\nSELECT DISTINCT * WHERE { ?tstat_setpoint_deadband rdf:type ns2:QuantifiableObservableProperty .\n?space_area ns4:hasQuantityKind ns3:Area .\n?tstat_active ns2:hasAspect ?tstat_active_aspects_in\n?tstat_active ns2:hasAspect ns2:Effectiveness-Active .\n?tstat_resolution ns4:hasUnit ?tstat_resolution_unit .\n?tstat_stage_count rdf:type ns2:EnumeratedObservableProperty .\n?hvac_cooling_COP ns4:hasUnit ?hvac_cooling_COP_unit .\n?window_area ns2:hasValue ?window_area_value .\n?window_tilt ns4:hasUnit ?window_tilt_unit .\n?hvac_cooling_COP ns4:hasQuantityKind ns3:CoefficientOfPerformance .\n?hvac ns2:hasProperty ?hvac_heating_capacity .\n?window_tilt rdf:type ns2:QuantifiableObservableProperty .\n?hvac_cooling_capacity rdf:type ns2:QuantifiableObservableProperty .\n?tstat_tolerance ns2:hasValue ?tstat_tolerance_value .\n?tstat_stage_count ns4:hasQuantityKind ns3:Count .\n?space_area ns4:hasUnit ?space_area_unit .\n?space ns2:hasProperty ?space_area .\n?window_azimuth ns2:hasValue ?window_azimuth_value .\n?tstat_setpoint_deadband ns4:isDeltaQuantity true .\n?space_area ns2:hasValue ?space_area_value .\n?hvac ns2:connectedTo ?name .\n?hvac_cooling_capacity ns4:hasQuantityKind ns3:Power .\n?window_azimuth ns4:hasUnit ?window_azimuth_unit .\n?hvac_heating_COP ns2:hasValue ?hvac_heating_COP_value .\n?tstat_active ns2:hasAspect ns2:Binary-Logical .\n?tstat_tolerance ns4:isDeltaQuantity true .\n?tstat_stage_count ns2:hasValue ?tstat_stage_count_value .\n?hvac_cooling_capacity ns2:hasAspect ?hvac_cooling_capacity_aspects_in\n?hvac_cooling_capacity ns2:hasAspect ns2:Role-Cooling .\n?space_area rdf:type ns2:QuantifiableObservableProperty .\n?hvac_heating_capacity ns4:hasQuantityKind ns3:Power .\n?hvac_cooling_COP ns2:hasAspect ?hvac_cooling_COP_aspects_in\n?hvac_cooling_COP ns2:hasAspect ns2:Role-Cooling .\n?window_tilt ns4:hasQuantityKind ns3:Tilt .\n?tstat_stage_count ns2:hasAspect ?tstat_stage_count_aspects_in\n?tstat_stage_count ns2:hasAspect ns2:Role-Cooling .\n?name ns2:encloses ?name .\n?tstat_active ns2:hasValue ?tstat_active_value .\n?tstat_tolerance ns2:hasAspect ?tstat_tolerance_aspects_in\n?tstat_tolerance ns2:hasAspect ns2:Aspect-Heating .\n?hvac ns2:hasProperty ?hvac_heating_COP .\n?tstat_resolution ns2:hasValue ?tstat_resolution_value .\n?tstat ns2:hasProperty ?tstat_resolution .\n?tstat_tolerance ns4:hasQuantityKind ns3:Temperature .\n?tstat_stage_count ns2:hasAspect ns2:Aspect-OperatingMode .\n?hvac_heating_COP ns4:hasQuantityKind ns3:CoefficientOfPerformance .\n?tstat_resolution rdf:type ns2:QuantifiableObservableProperty .\n?window_area ns4:hasUnit ?window_area_unit .\n?tstat rdf:type ns2:Thermostat .\n?name ns2:hasDomain ns2:Domain-HVAC .\n?tstat_active rdf:type ns2:EnumeratedObservableProperty .\n?hvac_heating_capacity ns4:hasUnit ?hvac_heating_capacity_unit .\n?hvac_heating_capacity ns2:hasValue ?hvac_heating_capacity_value .\n?window rdf:type ns2:Window .\n?hvac rdf:type ns2:AirToAirHeatPump .\n?tstat_tolerance ns4:hasUnit ?tstat_tolerance_unit .\n?tstat ns2:hasProperty ?tstat_tolerance .\n?tstat_tolerance ns2:hasAspect ns2:Aspect-Cooling .\n?tstat_tolerance ns2:hasAspect ns2:Aspect-Threshold .\n?tstat_active ns2:hasAspect ns2:Aspect-Heating .\n?hvac_cooling_COP rdf:type ns2:QuantifiableObservableProperty .\n?tstat_setpoint_deadband ns2:hasValue ?tstat_setpoint_deadband_value .\n?window ns2:hasProperty ?window_area .\n?tstat ns2:hasProperty ?tstat_stage_count .\n?tstat ns2:hasProperty ?tstat_active .\n?hvac_cooling_COP ns2:hasValue ?hvac_cooling_COP_value .\n?window ns2:hasProperty ?window_tilt .\n?tstat_resolution ns4:isDeltaQuantity true .\n?tstat_active ns4:hasUnit ns5:NUM .\n?window_tilt ns2:hasValue ?window_tilt_value .\n?hvac_heating_capacity ns2:hasAspect ?hvac_heating_capacity_aspects_in\n?hvac_heating_capacity ns2:hasAspect ns2:Role-Heating .\n?hvac ns2:hasProperty ?hvac_cooling_capacity .\n?tstat_setpoint_deadband ns2:hasAspect ?tstat_setpoint_deadband_aspects_in\n?tstat_setpoint_deadband ns2:hasAspect ns2:Aspect-Deadband .\n?window ns2:hasProperty ?window_azimuth .\n?window_area ns4:hasQuantityKind ns3:Area .\n?window_azimuth rdf:type ns2:QuantifiableObservableProperty .\n?tstat_stage_count ns4:hasUnit ns5:NUM .\n?tstat_active ns2:hasAspect ns2:Aspect-Cooling .\n?hvac_heating_capacity rdf:type ns2:QuantifiableObservableProperty .\n?hvac_heating_COP ns2:hasAspect ?hvac_heating_COP_aspects_in\n?hvac_heating_COP ns2:hasAspect ns2:Role-Heating .\n?space rdf:type ns2:PhysicalSpace .\n?name rdf:type ns2:DomainSpace .\n?window_azimuth ns4:hasQuantityKind ns3:Azimuth .\n?tstat_setpoint_deadband ns4:hasUnit ?tstat_setpoint_deadband_unit .\n?hvac_cooling_capacity ns2:hasValue ?hvac_cooling_capacity_value .\n?name ns2:connectedTo ?window .\n?window_area rdf:type ns2:QuantifiableObservableProperty .\n?hvac ns2:hasProperty ?hvac_cooling_COP .\n?hvac_heating_COP ns4:hasUnit ?hvac_heating_COP_unit .\n?tstat_setpoint_deadband ns4:hasQuantityKind ns3:Temperature .\n?tstat ns2:hasProperty ?tstat_setpoint_deadband .\n?hvac_cooling_capacity ns4:hasUnit ?hvac_cooling_capacity_unit .\n?name ns2:contains ?space .\n?hvac_heating_COP rdf:type ns2:QuantifiableObservableProperty .\n?tstat_tolerance rdf:type ns2:QuantifiableObservableProperty .\n?name rdf:type ns2:PhysicalSpace .\n?tstat ns2:hasMeasurementLocation ?name .\nFILTER(?tstat_active_aspects_in IN (ns2:Effectiveness-Active,ns2:Aspect-Heating,ns2:Binary-Logical,ns2:Aspect-Cooling) ) .\nFILTER(?hvac_cooling_capacity_aspects_in IN (ns2:Role-Cooling) ) .\nFILTER(?hvac_cooling_COP_aspects_in IN (ns2:Role-Cooling) ) .\nFILTER(?tstat_stage_count_aspects_in IN (ns2:Role-Cooling,ns2:Aspect-OperatingMode) ) .\nFILTER(?tstat_tolerance_aspects_in IN (ns2:Aspect-Cooling,ns2:Aspect-Threshold,ns2:Aspect-Heating) ) .\nFILTER(?hvac_heating_capacity_aspects_in IN (ns2:Role-Heating) ) .\nFILTER(?tstat_setpoint_deadband_aspects_in IN (ns2:Aspect-Deadband) ) .\nFILTER(?hvac_heating_COP_aspects_in IN (ns2:Role-Heating) ) . }'
query_to_df(query, loader.g)

SyntaxError: error at 80:9: expected OPTIONAL (<string>)

In [ ]:
print(zone.tstats[0].tstat_resolution)
zone.tstats[0].tstat_resolution.convert_to_si()
print(zone.tstats[0].tstat_resolution)
print(zone.tstats[0].tstat_resolution.is_delta)

None


AttributeError: 'NoneType' object has no attribute 'convert_to_si'

In [ ]:
# optionally just load everything as si 
si_loader = LoadModel(f"{ontology}-test_site/test_build/test_build.ttl", ontology = 'brick', as_si_units=True)
site_info = si_loader.get_all_building_objects()
print(zone.tstats[0].resolution)

Value(value=1.6666666666666667, unit='http://qudt.org/vocab/unit/DEG_C')


In [ ]:
# Getting the thermostat metadata
get_thermostat_data(si_loader)

{'heat_availability': [],
 'cool_availability': [],
 'heat_tolerance': [],
 'cool_tolerance': [],
 'setpoint_deadband': [],
 'active': [],
 'control_group': [],
 'control_type_list': [],
 'floor_area_list': [],
 'floor_area_unit': [],
 'window_area_list': [],
 'window_area_unit': [],
 'azimuth_list': [],
 'azimuth_unit': [],
 'tilt_list': [],
 'tilt_unit': [],
 'zone_ids': [],
 'hvacs': [],
 'setpoint_type': [],
 'fuel_heat_list': [],
 'fuel_cool_list': [],
 'cooling_capacity': [],
 'cooling_capacity_unit': [],
 'heating_capacity': [],
 'heating_capacity_unit': [],
 'cooling_cop': [],
 'heating_cop': [],
 'cooling_electricity': [],
 'heating_electricity': [],
 'resolution': [],
 'temperature_unit': []}

In [ ]:
# Getting the thermostat metadata for 1 zone 
get_thermostat_data(si_loader, ['zone_1','zone_2'])

{'heat_availability': [],
 'cool_availability': [],
 'heat_tolerance': [],
 'cool_tolerance': [],
 'setpoint_deadband': [],
 'active': [],
 'control_group': [],
 'control_type_list': [],
 'floor_area_list': [],
 'floor_area_unit': [],
 'window_area_list': [],
 'window_area_unit': [],
 'azimuth_list': [],
 'azimuth_unit': [],
 'tilt_list': [],
 'tilt_unit': [],
 'zone_ids': [],
 'hvacs': [],
 'setpoint_type': [],
 'fuel_heat_list': [],
 'fuel_cool_list': [],
 'cooling_capacity': [],
 'cooling_capacity_unit': [],
 'heating_capacity': [],
 'heating_capacity_unit': [],
 'cooling_cop': [],
 'heating_cop': [],
 'cooling_electricity': [],
 'heating_electricity': [],
 'resolution': [],
 'temperature_unit': []}

# Testing Unit Conversion

In [ ]:
convert_units(10, 'FT', 'M')

3.048

In [ ]:
convert_units(0, 'DEG_C', 'DEG_F')

31.999999999999886

In [ ]:
convert_units(0, 'DEG_C', 'K')

273.15

In [ ]:
# Planning to implement converting everything to si in loader
# loader.get_all_building_objects

# Testing Grafana Dashboarding 
Needs updates

In [ ]:
# import yaml 
# with open('../development_files/creds.yml') as f:
#     config = yaml.safe_load(f)

# bg = BrickToGrafana(grafana_server=config['grafana_server'], grafana_api_key = config['grafana_api_key'], datasource=config['datasource'], ttl_path = 'test-brick-model.ttl')

In [ ]:
# bg.create_dashboard('AQL from brick')

In [ ]:
# Grafana not set up, upload won't work
# bg.upload_dashboard(message = 'testing upload')